> 文章 ID：A007
>
> 创建时间：2023-06-27
>
> 更新时间：2023-06-26

> 本文档测试版本基于STIX2.1
> 
> MITRE ATTACK 矩阵的仓库地址和使用方法见如下链接：
> 
> 仓库地址：https://github.com/mitre-attack/attack-stix-data
> 
> 使用方法：https://github.com/mitre-attack/attack-stix-data/blob/master/USAGE.md
>
> 本项目已从仓库中拉下来了原始数据，见路径：../assets/attck_database/attack-stix-data

> 本文档采用python的 `stix2` 和 `mitreattack-python` 库去处理Mitre Attack的STIX格式数据

> 更多参考文档
> 
> https://zhuanlan.zhihu.com/p/311145611?utm_id=0

# ATT&CK 数据解析

![ATT&CK数据组织结构](../assets/article_data/A007/pic/P001.jpg)

ATT&CK到STIX2对象的映射


|ATT&CK概念	|STIX对象类型|
|---|---|
|Matrix	|x-mitre-matrix 自定义类型|
|Tactic	|x-mitre-tactic 自定义类型|
|Technique	|attack-pattern|
|Sub-technique	|attack-pattern (x_mitre_is_subtechnique = true)|
|Procedure	|relationship(relationship_type = "uses" 且 target_ref 是 attack-pattern)|
|Mitigation	|course-of-action|
|Group	|intrusion-set|
|Software	|malware 或 tool|


使用到的其他对象类型

|STIX 对象类型	| 描述|
|---|---|
|identity	| 在所有对象的created_by_ref中引用，声明MITRE公司创建的对象|
|marking-definition	| 在所有对象的object_marking_refs中引用，表示的版权|



STIX ID 用于标识对象的唯一编码，STIX ID是以编程方式检索和引用对象的最佳方法。例子：``` "id": "intrusion-set--f40eb8ce-2a74-4e56-89a1-227021410142" ```

### **Matrices**

ATT&CK矩阵总体布局存储在x-mitre-matrix中

扩展字段

| Field       | Type     | Description                                                  |
| ----------- | -------- | ------------------------------------------------------------ |
| tactic_refs | string[] | tactic_refs数组包含与矩阵的战术相对应的x-mitre-tactic STIX ID有序列表。 |




### **Tactics**

ATT&CK中的战术由x-mitre-tactic定义

扩展字段

| Field             | Type   | Description                                                  |
| ----------------- | ------ | ------------------------------------------------------------ |
| x_mitre_shortname | string | x_mitre_shortname用于将技术映射到战术中，对应技术中的kill_chain_phases.phase_name |





### **Techniques**

ATT&CK中的技术被定义为attack-pattern对象

扩展字段

| Field                        | Type     | Applies to                                           | Description                                                  |
| ---------------------------- | -------- | ---------------------------------------------------- | ------------------------------------------------------------ |
| x_mitre_detection            | string   | All techniques                                       | 识别攻击者是否使用了该技术的方法                             |
| x_mitre_platforms            | string[] | All techniques                                       | 适用于该技术的平台列表                                       |
| x_mitre_data_sources         | string[] | Enterprise and ICS domains                           | 识别动作或动作结果的信息源                                   |
| x_mitre_is_subtechnique      | boolean  | Enterprise domain                                    | 值为True时，代表该技术为子技术                               |
| x_mitre_system_requirements  | string   | Enterprise domain                                    | 攻击者需要满足的其他信息，或者该技术可能需要的系统状态信息（软件、补丁等） |
| x_mitre_tactic_types         | string   | Mobile domain                                        | "Post-Adversary Device Access", "Pre-Adversary Device Access", or "Without Adversary Device Access". |
| x_mitre_permissions_required | string[] | Enterprise domain in the Privilege Escalation tactic | 攻击者在系统上执行该技术需要的最低权限                       |
| x_mitre_defense_bypassed     | string[] | Enterprise domain in the Defense Evasion tactic      | 技术可以绕过的防御工具，方法或过程的列表                     |
| x_mitre_remote_support       | boolean  | Enterprise domain in the Execution tactic            | 值为True时，代表该技术可以在远程系统上执行操作               |

技术通过kill_chain_phases属性映射为战术

kill_chain_name可以是mitre-attack，mitre-mobile-attack或mitre-ics-attack（分别对应enterprise, mobile, and ics）

phase_name对应x-mitre-tactic对象的x_mitre_shortname属性





### **Sub-Techniques**

格式与Techniques相同，使用x_mitre_is_subtechnique=True来表示子技术

相关的关系(Relationships)类型subtechnique-of

source_ref给出子技术，target_ref给出父技术

```text
"source_ref": "attack-pattern--f3d95a1f-bba2-44ce-9af7-37866cd63fd0",
"target_ref": "attack-pattern--35dd844a-b219-4e2b-a6bb-efa9a75995a9",
"relationship_type": "subtechnique-of",
```

子技术仅存在于enterprise域中




### **Procedures**

ATT&CK中的过程被表示为relationship_type = "uses" 且target_ref为attack-pattern的关系

过程可以源于组(intrusion-set)和软件(malware, tool)的使用

过程的内容在description中描述




### **Mitigations**

ATT&CK中的缓解措施被定义为course-of-action对象，与STIX中的course-of-action相同

### **Groups**

ATT&CK中的组被定义为intrusion-set对象，与STIX中的ntrusion-set相同

### **Software**

ATT&CK中的软件是malware和tool对象的结合

扩展字段

| Field             | Type     | Description            |
| ----------------- | -------- | ---------------------- |
| x_mitre_platforms | string[] | 适用于该软件的平台列表 |
| x_mitre_aliases   | string[] | 给定软件的别名列表     |





### **Relationships**

ATT&CK中的对象通过relationship对象相互关联

| Source Type      | Relationship Type | Target Type    | Custom Type? | About                                                        |
| ---------------- | ----------------- | -------------- | ------------ | ------------------------------------------------------------ |
| intrusion-set    | uses              | malware或tool  | No           | Group使用software                                            |
| intrusion-set    | uses              | attack-pattern | No           | Group使用technique(也被视为procedure example)                |
| malware或tool    | uses              | attack-pattern | No           | Software使用technique(也被视为procedure example)             |
| course-of-action | mitigates         | attack-pattern | No           | Mitigation缓解technique                                      |
| attack-pattern   | subtechnique-of   | attack-pattern | Yes          | 技术的子技术                                                 |
| any type         | revoked-bya       | any type       | Yes          | target对象是source对象的替代（两者具有相同类型，且source对象属性revoked = true时）（source对象已被撤销） |

# Python 数据样例

In [1]:
import mitreattack
from stix2 import MemoryStore
from stix2 import Filter


In [2]:
# 将attack-stix-data数据载入缓存
src = MemoryStore()
src.load_from_file("../assets/attck_database/attack-stix-data/enterprise-attack/enterprise-attack.json")

In [5]:
import os
# 载入特定版本的数据
def get_attack_version(root,domain, version):
    """get ATT&CK STIX data for a given domain and version. Domain should be 'enterprise-attack', 'mobile-attack' or 'ics-attack'. Branch should typically be master."""
    ms = MemoryStore()
    print(os.path.join(root,domain, f"{domain}-{version}.json"))
    ms.load_from_file(os.path.join(root,domain, f"{domain}-{version}.json"))
    return ms

src = get_attack_version("../assets/attck_database/attack-stix-data","enterprise-attack", "6.2")

../assets/attck_database/attack-stix-data\enterprise-attack\enterprise-attack-6.2.json


以G0075为例，来查询对象，其json数据如下：

In [ ]:
{
    "type": "intrusion-set",
    "spec_version": "2.1",
    "id": "intrusion-set--f40eb8ce-2a74-4e56-89a1-227021410142",
    "created_by_ref": "identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5",
    "created": "2018-10-17T00:14:20.652Z",
    "modified": "2022-04-25T14:00:00.188Z",
    "name": "Rancor",
    "description": "[Rancor](https://attack.mitre.org/groups/G0075) is a threat group that has led targeted campaigns against the South East Asia region. [Rancor](https://attack.mitre.org/groups/G0075) uses politically-motivated lures to entice victims to open malicious documents. (Citation: Rancor Unit42 June 2018)",
    "aliases": [
        "Rancor"
    ],
    "external_references": [
        {
            "source_name": "mitre-attack",
            "url": "https://attack.mitre.org/groups/G0075",
            "external_id": "G0075"
        },
        {
            "source_name": "Rancor",
            "description": "(Citation: Rancor Unit42 June 2018)"
        },
        {
            "source_name": "Rancor Unit42 June 2018",
            "description": "Ash, B., et al. (2018, June 26). RANCOR: Targeted Attacks in South East Asia Using PLAINTEE and DDKONG Malware Families. Retrieved July 2, 2018.",
            "url": "https://researchcenter.paloaltonetworks.com/2018/06/unit42-rancor-targeted-attacks-south-east-asia-using-plaintee-ddkong-malware-families/"
        }
    ],
    "object_marking_refs": [
        "marking-definition--fa42a846-8d90-4e51-bc29-71d5b4802168"
    ],
    "x_mitre_attack_spec_version": "2.1.0",
    "x_mitre_domains": [
        "enterprise-attack"
    ],
    "x_mitre_modified_by_ref": "identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5",
    "x_mitre_version": "1.2"
}

In [4]:
# 通过 STIX ID 查询对象
g0075 = src.get("intrusion-set--f40eb8ce-2a74-4e56-89a1-227021410142")
print(g0075)

{"type": "intrusion-set", "spec_version": "2.1", "id": "intrusion-set--f40eb8ce-2a74-4e56-89a1-227021410142", "created_by_ref": "identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5", "created": "2018-10-17T00:14:20.652Z", "modified": "2022-04-25T14:00:00.188Z", "name": "Rancor", "description": "[Rancor](https://attack.mitre.org/groups/G0075) is a threat group that has led targeted campaigns against the South East Asia region. [Rancor](https://attack.mitre.org/groups/G0075) uses politically-motivated lures to entice victims to open malicious documents. (Citation: Rancor Unit42 June 2018)", "aliases": ["Rancor"], "external_references": [{"source_name": "mitre-attack", "url": "https://attack.mitre.org/groups/G0075", "external_id": "G0075"}, {"source_name": "Rancor", "description": "(Citation: Rancor Unit42 June 2018)"}, {"source_name": "Rancor Unit42 June 2018", "description": "Ash, B., et al. (2018, June 26). RANCOR: Targeted Attacks in South East Asia Using PLAINTEE and DDKONG Malware Famili

In [8]:
# 通过 ATT&CK ID 查询对象
g0075 = src.query([ Filter("external_references.external_id","=","G0075")])[0]
print(g0075)

{"type": "intrusion-set", "spec_version": "2.1", "id": "intrusion-set--f40eb8ce-2a74-4e56-89a1-227021410142", "created_by_ref": "identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5", "created": "2018-10-17T00:14:20.652Z", "modified": "2022-04-25T14:00:00.188Z", "name": "Rancor", "description": "[Rancor](https://attack.mitre.org/groups/G0075) is a threat group that has led targeted campaigns against the South East Asia region. [Rancor](https://attack.mitre.org/groups/G0075) uses politically-motivated lures to entice victims to open malicious documents. (Citation: Rancor Unit42 June 2018)", "aliases": ["Rancor"], "external_references": [{"source_name": "mitre-attack", "url": "https://attack.mitre.org/groups/G0075", "external_id": "G0075"}, {"source_name": "Rancor", "description": "(Citation: Rancor Unit42 June 2018)"}, {"source_name": "Rancor Unit42 June 2018", "description": "Ash, B., et al. (2018, June 26). RANCOR: Targeted Attacks in South East Asia Using PLAINTEE and DDKONG Malware Famili

In [10]:
glist = src.query([ Filter("external_references.external_id","=","G.*")])
print(glist)

[]
